In [69]:
import pandas as pd
import os
import datetime
import numpy as np

In [70]:
#this is an old function we are not using this anymore
def getMeanXYZ(df):
    if df.empty:
        return 
    #removing excess data so that data falls in 1 second data intervals
    maxIndx = int(len(df)/100)*100
    df = df.iloc[0:maxIndx]
    meanPerSecondDf = pd.DataFrame()
    
    for i in range(0,int(maxIndx/100)):
        startIndx = i*100
        endIndx = startIndx+100
        tempdf = df.iloc[startIndx:endIndx]
        
        meanDf = tempdf.mean(axis=0)
        stdDf = tempdf.std()
        
        meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'mean_y':meanDf.Y,'std_y':stdDf.Y, 'mean_z':meanDf.Z, 'std_z':stdDf.Z},ignore_index=True)
        
    return meanPerSecondDf


In [78]:
def getMeanXYZ_updated(df):
    startInd = 0
    endInd = 1

    meanPerSecondDf = pd.DataFrame()
    updateFlag = False
    
    #running this for loop for the length of the dataframe
    for i in range(0, len(df)):
        """As we are getting in seconds rather than milliseconds
        We confrimed this using https://www.epochconverter.com"""
        #Setting updateFlag to indicated window for data
        if(df.iloc[i].time - df.iloc[startInd].time > 1000):
            updateFlag = True

        #taking the window and calculating std and mean based in window
        if updateFlag==True:
            updateFlag = False
            tempdf = df.iloc[startInd:i-1]
            startInd = i

            meanDf = tempdf.mean(axis=0)
            stdDf = tempdf.std()
            meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'mean_y':meanDf.Y,'std_y':stdDf.Y, 'mean_z':meanDf.Z, 'std_z':stdDf.Z},ignore_index=True)
    return meanPerSecondDf

In [79]:
#trying out stuff

#need to do this for all files
#reading the file with no header
df = pd.read_csv("Data/G7NZCJ008056297-Justin-left-hand_wash--1970-01-01-00-59-42.csv", header=None)
#renaming the columns
df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')
meanDf = getMeanXYZ(df)
df

,time,X,Y,Z
0,21454957,-1.175955,3.323090,8.359816
1,21454967,-1.179547,3.494334,8.297545
2,21455072,-0.275427,2.511178,9.007668
3,21455077,-0.173639,2.617756,8.926238
4,21455087,-0.189207,2.537523,9.007668
...,...,...,...,...
778,21463002,-2.372267,2.240541,8.429272
779,21463012,-2.573448,2.116000,8.416099
780,21463022,-2.756667,1.986669,8.515491
781,21463032,-2.988984,2.181863,8.344248


In [80]:
"""
In this cell trying to figure out how to write the function: getMeanXYZ_updated
"""

# temp = pd.to_datetime(df['time'].astype('str'), format ="%d/%m %H:%M:%S")
temp = df.time
tempDf = pd.DataFrame()

startInd = 0
endInd = 1

meanPerSecondDf = pd.DataFrame()

updateFlag = False
for i in range(0, len(df)):
    if(df.iloc[i].time - df.iloc[startInd].time > 60):
        updateFlag = True
    
    if updateFlag==True:
        updateFlag = False
        tempdf = df.iloc[startInd:i-1]
        startInd = i
        
        meanDf = tempdf.mean(axis=0)
        stdDf = tempdf.std()
        meanPerSecondDf = meanPerSecondDf.append({'mean_x':meanDf.X, 'std_x':stdDf.X, 'mean_y':meanDf.Y,'std_y':stdDf.Y, 'mean_z':meanDf.Z, 'std_z':stdDf.Z},ignore_index=True)
meanPerSecondDf

,mean_x,std_x,mean_y,std_y,mean_z,std_z
0,-1.175955,NaN,3.323090,NaN,8.359816,NaN
1,-0.273232,0.090832,2.524550,0.056997,8.912466,0.080559
2,-0.444276,0.130098,2.394421,0.149198,8.747410,0.497712
3,-1.386716,0.499137,1.396496,1.386063,6.473738,0.529306
4,-0.326920,0.273794,-4.757706,1.973065,3.909470,0.187076
...,...,...,...,...,...,...
111,-2.874023,0.129025,1.831791,0.313955,7.971623,0.094056
112,-2.581352,0.086911,1.876737,0.220258,8.058283,0.130200
113,-2.638912,0.131390,2.037164,0.346737,8.420490,0.112269
114,-2.155518,0.932770,1.760140,0.965102,8.746013,0.809792


In [81]:
#main code
dataPath = 'Data/'
handWashSubString = 'left-hand_wash'
notHandWashSubstring = 'left-not_hand_wash'

handWashedDf = pd.DataFrame()
notHandWashedDF = pd.DataFrame()

#Reading through the data folder for filenames
for path in os.listdir(dataPath):
    filepath = dataPath+path
    df = pd.read_csv(filepath, header=None)
    df = df.rename({0:'time', 1: 'X', 2: 'Y', 3: 'Z'},axis='columns')
    
    meanDf = getMeanXYZ_updated(df)   #using the updated getMeanXYZ function
        
    if handWashSubString in path:
        handWashedDf = handWashedDf.append(meanDf,ignore_index=True)
    else:
        notHandWashedDF = notHandWashedDF.append(meanDf,ignore_index=True)

In [82]:
handWashedDf['Activity'] = 'hand_wash'
handWashedDf

,mean_x,std_x,mean_y,std_y,mean_z,std_z,Activity
0,1.652931,3.442638,-5.572326,6.064842,-0.031856,7.454768,hand_wash
1,-1.807909,3.107468,-9.217029,3.334817,-3.186268,5.243445,hand_wash
2,-1.296940,2.364967,-9.333873,3.010561,-2.229245,4.290689,hand_wash
3,-1.321506,0.646286,-9.476633,0.899180,-2.606725,0.862985,hand_wash
4,-1.377510,1.219219,-9.081902,2.117393,-1.758705,3.397296,hand_wash
...,...,...,...,...,...,...,...
135,1.339545,2.766707,-8.236811,4.686479,-0.263830,5.806000,hand_wash
136,1.484184,3.408315,-9.233282,4.777810,-3.709898,8.527217,hand_wash
137,1.205917,4.106732,-7.536151,6.240607,4.541784,4.721256,hand_wash
138,0.771508,5.180624,-8.073798,7.385673,-1.185185,7.602765,hand_wash


In [83]:
notHandWashedDF['Activity'] = 'no_hand_wash'
notHandWashedDF

,mean_x,std_x,mean_y,std_y,mean_z,std_z,Activity
0,-1.454442,2.123706,-1.660613,7.122687,6.203313,5.258093,no_hand_wash
1,0.498028,17.184565,-3.896339,12.259085,1.950163,5.631699,no_hand_wash
2,2.474503,13.943652,-2.656911,11.105086,1.911528,6.407991,no_hand_wash
3,-1.600996,9.687005,-2.097563,9.669482,-1.372334,16.631289,no_hand_wash
4,-0.012290,10.889246,6.838712,4.953479,3.924131,5.137993,no_hand_wash
...,...,...,...,...,...,...,...
355,5.533239,14.328041,1.929882,15.547365,-0.020017,7.301156,no_hand_wash
356,7.636181,12.836038,4.101955,12.189794,4.459284,8.174824,no_hand_wash
357,1.154762,20.572129,13.743348,17.464195,5.574227,13.963414,no_hand_wash
358,4.055239,18.315336,4.166200,16.059836,2.329742,10.986047,no_hand_wash


In [85]:
nameOfTheFile = 'processedDataAssignment1.csv'
to_save_df = pd.DataFrame()
to_save_df = to_save_df.append(handWashedDf,ignore_index=True)
to_save_df = to_save_df.append(notHandWashedDF,ignore_index=True)

#removing data points that are empty
to_save_df['std_x'].replace('', np.nan, inplace=True)
to_save_df.dropna(subset=['std_x'], inplace=True)

#saving the dataframe to a file as defined in nameOfTheFile
to_save_df.to_csv(nameOfTheFile, index=False,encoding='utf-8')